In [11]:
import numpy as np
import cv2
from pathlib import Path

# --- CONFIGURATION ---
FILE_PATH = "../experiments/generated_output.npz"
OUTPUT_VIDEO = "skeleton_check.mp4"
WIDTH, HEIGHT = 1000, 1000
FPS = 60

# MediaPipe Hand Connections
HAND_CONNECTIONS = [
    (0, 1), (1, 2), (2, 3), (3, 4),      # Thumb
    (0, 5), (5, 6), (6, 7), (7, 8),      # Index
    (5, 9), (9, 10), (10, 11), (11, 12), # Middle
    (9, 13), (13, 14), (14, 15), (15, 16), # Ring
    (0, 17), (17, 18), (18, 19), (19, 20)  # Pinky
]

def draw_generated_hands_only(data_path, output_path):
    data = np.load(data_path)
    lh = data['lh']  # (Frames, 21, 3)
    rh = data['rh']  # (Frames, 21, 3)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, FPS, (WIDTH, HEIGHT))

    # --- ADJUSTED SCALE ---
    # Increase this number if the hands still look like dots
    VISUAL_SCALE = 6000 
    
    # Fixed screen positions for the hands
    LEFT_ANCHOR  = (WIDTH // 3, HEIGHT // 2)
    RIGHT_ANCHOR = (2 * WIDTH // 3, HEIGHT // 2)

    print(f"🎬 Generating hand-only video for {len(lh)} frames...")

    for i in range(len(lh)):
        # Black background
        frame = np.ones((HEIGHT, WIDTH, 3), dtype=np.uint8) * 20

        for hand_pts, anchor, color in [
            (lh[i], LEFT_ANCHOR, (255, 100, 0)), # Cyan-ish
            (rh[i], RIGHT_ANCHOR, (0, 0, 255))   # Red
        ]:
            # Skip if the hand wasn't generated (all zeros)
            if np.all(np.abs(hand_pts) < 1e-5):
                continue

            # 1. Reverse the 5x scaling from training
            hand = hand_pts / 5.0
            
            # 2. Draw Bones
            for s, e in HAND_CONNECTIONS:
                p1 = (
                    int(hand[s][0] * VISUAL_SCALE + anchor[0]),
                    int(hand[s][1] * VISUAL_SCALE + anchor[1])
                )
                p2 = (
                    int(hand[e][0] * VISUAL_SCALE + anchor[0]),
                    int(hand[e][1] * VISUAL_SCALE + anchor[1])
                )
                cv2.line(frame, p1, p2, color, 3, cv2.LINE_AA)

            # 3. Draw Joint Dots
            for pt in hand:
                px = int(pt[0] * VISUAL_SCALE + anchor[0])
                py = int(pt[1] * VISUAL_SCALE + anchor[1])
                cv2.circle(frame, (px, py), 4, (255, 255, 255), -1, cv2.LINE_AA)

        # Frame counter
        cv2.putText(frame, f"Frame: {i}", (30, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (150, 150, 150), 1)

        out.write(frame)

    out.release()
    print(f"✅ Hand-only video saved: {output_path}")

# Run the script
draw_generated_hands_only(FILE_PATH, OUTPUT_VIDEO)

🎬 Generating hand-only video for 75 frames...
✅ Hand-only video saved: skeleton_check.mp4


NameError: name 'generator' is not defined